<a href="https://colab.research.google.com/github/LipsaBiswas/Summary-comparer/blob/main/GradioTestApp2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# use the configuration file requirements.txt to install the specified packages with the specified version.
# !pip install -q -r requirements.txt

     |████████████████████████████████| 1.5MB 4.3MB/s 
     |████████████████████████████████| 2.3MB 36.4MB/s 
     |████████████████████████████████| 1.2MB 32.1MB/s 
     |████████████████████████████████| 215kB 34.5MB/s 
     |████████████████████████████████| 1.9MB 36.3MB/s 
     |████████████████████████████████| 901kB 38.4MB/s 
     |████████████████████████████████| 3.3MB 35.5MB/s 
     |████████████████████████████████| 962kB 35.5MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 3.2MB 35.9MB/s 


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM #Huggingface transformer
import torch

#For paraphrasing
from nltk.tokenize import sent_tokenize
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import nltk
nltk.download('punkt')
#For interface
import gradio as gr


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
 device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
paraphrase_model_name = 'tuner007/pegasus_paraphrase' 
paraphrase_tokenizer = PegasusTokenizer.from_pretrained(paraphrase_model_name)
paraphrase_model = PegasusForConditionalGeneration.from_pretrained(paraphrase_model_name).to(device)

In [15]:
#Create a function to generate abstract summary based on input text and pre-trained model provided
'''
Function name : GenerateSummary
Input params :
      inp - abstract text whose summary will be generated
      pretrained_model_selected - pretrained model name
      max_pct - max percent of Original text to be the cap for summary length
'''
def GenerateSummary(inp,pretrained_model_selected):
  paraphrased_abstract=""
  paraphrased_summary=""
  
  try:
    pretrained_model_selected = pretrained_model_selected.lower() #lowercase the selected model name
    if pretrained_model_selected=="bert":
        from summarizer import Summarizer 
        bert_model=Summarizer()
        output = bert_model(inp )
        #PARAPHRASE abstract         
        paraphrased_abstract = ParaphrasePara(inp)
        #PARAPHRASE SUMMARY       
        paraphrased_summary = ParaphrasePara(output)
    else:
      
      #create auto tokenizer object
      tokenizer = AutoTokenizer.from_pretrained(pretrained_model_selected)
      #create model object
      model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_selected)
      #Save model to device
    
      model = model.to(device)

      #Preprocess the input text
      preprocess_text = inp.strip().replace("\n","")
      tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
      #input ids
      summary_ids = model.generate(
                  tokenized_text,
                  max_length=512, 
                  num_beams=5,
                  repetition_penalty=2.5, 
                  length_penalty=1.0, 
                  early_stopping=False
              )

      #SUMMARY of abstract
      output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
      #PARAPHRASE abstract      
      paraphrased_abstract = ParaphrasePara(inp)
      #PARAPHRASE SUMMARY      
      paraphrased_summary = ParaphrasePara(output)

  except Exception as e:
        output=" Error occured, please check with the admin: "+ str(e)
  return output,paraphrased_summary,paraphrased_abstract
  # print ("\n\nSummarized text: \n",output , "\n\nParaphrazed abstract: \n",paraphrased_abstract,"\n\nParaphrazed summary: \n",paraphrased_summary)

In [9]:
#Create a function to paraphrase sentences
'''
Function name - ParaphrasePara
input param - (string) sentences
'''
def ParaphrasePara(sentences):
  sentences=sentences.replace('.' , '.  ') #some issue with sent tokenizer. Its not tokenizing when no space after full stop
  paraphrase = []
  
  sentence_list = sent_tokenize(sentences) #Tokenize the input paragraph into different sentences
  # sentence_list
  for i in sentence_list:
    a = get_response(i,1) #Call get_response function for each sentence in the paragraph
    paraphrase.append(a)
  
  #Stylize the output
  paraphrase2 = [' '.join(x) for x in paraphrase]
  paraphrase3 = [' '.join(x for x in paraphrase2) ]
  paraphrased_text = str(paraphrase3).strip('[]').strip("'")
  # print(paraphrase)
  return paraphrased_text

In [11]:
#Create a function to paraphrase sentences using pretrained paraphrasing models

def get_response(input_text,num_return_sequences):  

  batch = paraphrase_tokenizer([input_text],truncation=True,padding='longest',  return_tensors="pt").to(device)
  translated = paraphrase_model.generate(**batch,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  text_generated = paraphrase_tokenizer.batch_decode(translated, skip_special_tokens=True)
  return text_generated

In [17]:
#Create GRADIO interface
iface = gr.Interface(
  fn=GenerateSummary, 
  inputs=[gr.inputs.Textbox(label="Abstract (*Enter paragraphs)",lines=20, placeholder="Enter abstract text here...") ,
          gr.inputs.Radio(["sshleifer/distilbart-cnn-12-6","deep-learning-analytics/wikihow-t5-small","Bert", "google/pegasus-pubmed", "mayu0007/pegasus_large_covid","google/pegasus-wikihow","google/pegasus-xsum"])
        ],    
  outputs=[
           gr.outputs.Textbox(label=" Summary"),
           gr.outputs.Textbox(label=" Paraphrased Summary "),
           gr.outputs.Textbox(label=" Paraphrased abstract") ],title= "Auto summarization and paraphrasing"  )
iface.launch(debug=False)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://28849.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7863/',
 'https://28849.gradio.app')

In [ ]:
# #Testing
# a ='''
# Severe Acute Respiratory Syndrome Coronavirus-2 (SARS-CoV-2), causing Coronavirus Disease 19 (COVID-19), 
# emerged at the end of 2019 and quickly spread to cause a global pandemic with severe socio-economic consequences. 
# The early sequencing of its RNA genome revealed its high similarity to SARS, likely to have originated from bats. 
# The SARS-CoV-2 non-structural protein 10 (nsp10) displays high sequence similarity with its SARS homologue, 
# which binds to and stimulates the 3'-to-5' exoribonuclease and the 2'-O-methlytransferase activities of nsps 14 and 16,
#  respectively. Here, we report the biophysical characterization and 1.6 Å resolution structure of the unbound
#   form of nsp10 from SARS-CoV-2 and compare it to the structures of its SARS homologue and the complex-bound 
#   form with nsp16 from SARS-CoV-2. The crystal structure and solution behaviour of nsp10 will not only form 
#   the basis for understanding the role of SARS-CoV-2 nsp10 as a central player of the viral RNA capping apparatus, but will also serve as a basis for the development of inhibitors of nsp10, 
# interfering with crucial functions of the replication-transcription complex and virus replication
# '''
# b='''
# The coronavirus (CoV) spike (S) protein, involved in viral-host cell fusion, is the primary immunogenic target for virus neutralization and the current focus of many vaccine design efforts. The highly flexible S-protein, with its mobile domains, presents a moving target to the immune system. Here, to better understand S-protein mobility, we implemented a structure-based vector analysis of available β-CoV S-protein structures. Despite an overall similarity in domain organization, we found that S-proteins from different β-CoVs display distinct configurations. Based on this analysis, we developed two soluble ectodomain constructs for the SARS-CoV-2 S-protein, in which the highly immunogenic and mobile receptor binding domain (RBD) is either locked in the all-RBDs 'down' position or adopts 'up' state conformations more readily than the wild-type S-protein. These results demonstrate that the conformation of the S-protein can be controlled via rational design and can provide a framework for the development of engineered CoV S-proteins for vaccine applications.
# '''

# c='''
# The COVID-19 pandemic has urged for the repurposing of existing drugs for rapid management and treatment. Renin inhibitors down regulation of ACE2, which is an essential receptor for SARS-CoV-2 infection that is responsible for COVID-19, in addition to their ability to act as protease inhibitors were encouraging aspects for their investigation as possible inhibitors of main protease of SARS-CoV-2 via computational studies. A Pharmacophore model was generated using the newly released SARS-COV-2 main protease inhibitors. Virtual screening was performed on renin inhibitors, and Drug likeness filter identified remikiren and 0IU as hits. Molecular docking for both compounds showed that the orally active renin inhibitor remikiren (Ro 42–5892) of Hoffmann–La Roche exhibited good molecular interaction with Cys145 and His41 in the catalytic site of SARS-CoV-2 main protease. Molecular dynamics simulation suggested that the drug is stable in the active site of the enzyme.
# '''
# GenerateSummary(c,'deep-learning-analytics/wikihow-t5-small')

# GenerateSummary(c,'sshleifer/distilbart-cnn-12-6')